In [2]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import os
import numpy as np

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


# General init (to delete what's not needed)

In [3]:
# Names and colors

colors_dict = {"Oncodrive3D" : "#ffad48", 
              "HotMAPS" : "#024b79", 
              "OncodriveCLUSTL" : "#43b7c2", 
              "smRegions" : "lightgray", 
              "CBaSE" : "lightgray", 
              "dNdScv" : "lightgray", 
              "MutPanning" : "lightgray", 
              "OncodriveFML" : "lightgray",
              "Random" : "pink"}

names_dict = {"o3d" : "Oncodrive3D", 
             "hotmaps" : "HotMAPS", 
             "clustl" : "OncodriveCLUSTL", 
             "smreg" : "smRegions", 
             "cbase" : "CBaSE", 
             "dndscv" : "dNdScv", 
             "mutpan" : "MutPanning", 
             "fml" : "OncodriveFML",
             "random" : "Random"}

In [4]:
SUBDIR = "human_mane_raw"
RUN = "run_2024-07-01_16-04-14"
# lst_methods = ["o3d", "hotmaps", "clustl", "smreg", "cbase", "seismic", "dndscv", "mutpan", "fml", "random"]
# lst_methods = ["o3d", "hotmaps", "clustl", "random"]

datasets = f"/workspace/projects/clustering_3d/o3d_analysys/datasets/"
input_path = f"{datasets}/input/cancer_202404"
cohort_df = pd.read_csv(f"{input_path}/cohorts.tsv", sep="\t")
cohort_df = cohort_df[~cohort_df.COHORT.str.startswith("ICGC")].reset_index(drop=True)
o3d_output_dir = f"{datasets}/output/cancer_202404/o3d_output/{SUBDIR}/{RUN}"
intogen_output = f"{datasets}/output/cancer_202404/intogen_output"
genetable_path = f"/workspace/projects/clustering_3d/o3d_analysys/o3d_paper/data/results/{SUBDIR}/{RUN}.genetable.tsv"
drivers_df = pd.read_table(f"{input_path}/drivers.tsv")
uniq_drivers_df = pd.read_table(f"{input_path}/unique_drivers.tsv")

cgc_df = pd.read_csv(f"{input_path}/CGC_list_v99.tsv", sep = "\t")
display(cgc_df)

oncotree_df = pd.read_csv(f"{datasets}/input/cancer_202404/oncotree.tsv", sep="\t")
display(oncotree_df)

oncokb_df = pd.read_csv(f"{input_path}/oncokb_list_20240712.tsv", sep = "\t")
oncokb_df = oncokb_df[["Hugo Symbol", "OncoKB Annotated", "MSK-IMPACT", "MSK-HEME", "FOUNDATION ONE", "FOUNDATION ONE HEME", "Vogelstein"]]
oncokb_df

,Gene Symbol,Name,Entrez GeneId,Genome Location,Tier,Hallmark,Chr Band,Somatic,Germline,Tumour Types(Somatic),Tumour Types(Germline),Cancer Syndrome,Tissue Type,Molecular Genetics,Role in Cancer,Mutation Types,Translocation Partner,Other Germline Mut,Other Syndrome,Synonyms
0,A1CF,APOBEC1 complementation factor,29974.0,10:50799421-50885675,2,NaN,10q11.23,yes,NaN,melanoma,NaN,NaN,E,NaN,oncogene,Mis,NaN,NaN,NaN,"ACF,ACF64,ACF65,APOBEC1CF,ASP,CCDS73133.1,ENSG..."
1,ABI1,abl-interactor 1,10006.0,10:26746593-26860935,1,Yes,10p12.1,yes,NaN,AML,NaN,NaN,L,Dom,"TSG, fusion",T,KMT2A,NaN,NaN,"ABI-1,CCDS7150.1,E3B1,ENSG00000136754.17,NM_00..."
2,ABL1,v-abl Abelson murine leukemia viral oncogene h...,25.0,9:130713946-130885683,1,Yes,9q34.12,yes,NaN,"CML, ALL, T-ALL",NaN,NaN,L,Dom,"oncogene, fusion","T, Mis","BCR, ETV6, NUP214",NaN,NaN,"ABL,CCDS35165.1,ENSG00000097007.17,JTK7,NM_007..."
3,ABL2,"c-abl oncogene 2, non-receptor tyrosine kinase",27.0,1:179099327-179229601,1,NaN,1q25.2,yes,NaN,AML,NaN,NaN,L,Dom,"oncogene, fusion",T,ETV6,NaN,NaN,"ABLL,ARG,CCDS30947.1,ENSG00000143322.19,NM_007..."
4,ACKR3,atypical chemokine receptor 3,57007.0,2:236569641-236582358,1,Yes,2q37.3,yes,NaN,lipoma,NaN,NaN,M,Dom,"oncogene, fusion",T,HMGA2,NaN,NaN,"CCDS2516.1,CMKOR1,CXCR7,ENSG00000144476.5,GPR1..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
738,ZNF429,zinc finger protein 429,353088.0,19:21505564-21538078,2,NaN,19p12,yes,NaN,GBM,NaN,NaN,O,NaN,NaN,Mis,NaN,NaN,NaN,"CCDS42537.1,ENSG00000197013.9,NM_001001415.2,N..."
739,ZNF479,zinc finger protein 479,90827.0,7:57119614-57139864,2,NaN,7p11.2,yes,NaN,"lung cancer, bladder carcinoma, prostate carci...",NaN,NaN,E,NaN,NaN,Mis,NaN,NaN,NaN,"CCDS43590.1,ENSG00000185177.12,KR19,NM_033273...."
740,ZNF521,zinc finger protein 521,25925.0,18:25061926-25352152,1,NaN,18q11.2,yes,NaN,ALL,NaN,NaN,L,Dom,"oncogene, fusion",T,PAX5,NaN,NaN,"CCDS32806.1,EHZF,ENSG00000198795.10,Evi3,NM_01..."
741,ZNRF3,zinc and ring finger 3,84133.0,22:28883592-29057487,2,NaN,22q12.1,yes,NaN,"colorectal cancer, adrenocortical carcinoma, g...",NaN,NaN,E,NaN,TSG,"N, F, Mis",NaN,NaN,NaN,"BK747E2.3,CCDS56225.1,ENSG00000183579.15,FLJ22..."


,ID,PARENT,NAMES
0,AA,SOFT_TISSUE,Aggressive Angiomyxoma
1,AASTR,DIFG,Anaplastic Astrocytoma
2,ABC,DLBCLNOS,Activated B-cell Type
3,ABL,AMLNOS,Acute Basophilic Leukemia
4,ACA,ADRENAL_GLAND,Adrenocortical Adenoma
...,...,...,...
883,WDLS,LIPO,Well-Differentiated Liposarcoma
884,WDTC,THYROID,Well-Differentiated Thyroid Cancer
885,WM,LPL,Waldenstrom Macroglobulinemia
886,WPSCC,PSCC,Warty Penile Squamous Cell Carcinoma


,Hugo Symbol,OncoKB Annotated,MSK-IMPACT,MSK-HEME,FOUNDATION ONE,FOUNDATION ONE HEME,Vogelstein
0,ABL1,Yes,Yes,Yes,Yes,Yes,Yes
1,AKT1,Yes,Yes,Yes,Yes,Yes,Yes
2,ALK,Yes,Yes,Yes,Yes,Yes,Yes
3,AMER1,Yes,Yes,Yes,Yes,Yes,Yes
4,APC,Yes,Yes,Yes,Yes,Yes,Yes
...,...,...,...,...,...,...,...
1149,ZFP36L1,Yes,No,No,No,No,No
1150,ZFP36L2,Yes,No,No,No,No,No
1151,ZNF24,No,No,No,No,Yes,No
1152,ZNF331,No,No,No,No,No,No


In [5]:
genetable = pd.read_csv(genetable_path, sep="\t", dtype={1 : str, 2 : str, 3 : int})
genetable = genetable[[not cohort.startswith("ICGC") for cohort in genetable["Cohort"]]].reset_index(drop=True)
genetable_subset = genetable[genetable["Cohort"].str.startswith("TCGA")]
genetable_subset

,Gene,Uniprot_ID,o3d_status,CGC,Fish,o3d_rank,o3d_pval,o3d_qval,cbase_rank,cbase_pval,...,clustl_pval,clustl_qval,fml_rank,fml_pval,fml_qval,smreg_rank,smreg_pval,smreg_qval,Cancer,Cohort
3062778,NFE2L2,Q16236,Processed,1,0,0.0,0.0,0.0,6.0,0.000000,...,0.002892,0.380525,3.0,0.000001,0.001441,NaN,NaN,NaN,LUSC,TCGA_WXS_LUSC
3062779,TP53,K7PPA8,Processed,1,0,1.0,0.0,0.0,11.0,0.000000,...,0.000055,0.073971,1.0,0.000001,0.001441,0.0,9.534979e-56,1.202361e-51,LUSC,TCGA_WXS_LUSC
3062780,PIK3CA,P42336,Processed,1,0,2.0,0.0,0.0,53.0,0.000388,...,0.000004,0.007465,29.0,0.000310,0.148955,1.0,8.013395e-12,5.052445e-08,LUSC,TCGA_WXS_LUSC
3062781,CDKN2A,K7PML8,Processed,1,0,3.0,0.0,0.0,5.0,0.000000,...,0.000399,0.266155,5.0,0.000001,0.001441,NaN,NaN,NaN,LUSC,TCGA_WXS_LUSC
3062782,HRAS,P01112,Processed,1,0,4.0,0.0,0.0,990.0,0.033670,...,0.003380,0.380525,2313.0,0.129510,0.927878,2046.0,1.413890e-01,7.547557e-01,LUSC,TCGA_WXS_LUSC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4582141,XKRY2,NaN,NaN,0,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PGNG,TCGA_WXS_PGNG
4582142,BCORL2,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PGNG,TCGA_WXS_PGNG
4582143,CYorf15A,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PGNG,TCGA_WXS_PGNG
4582144,CYorf15B,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PGNG,TCGA_WXS_PGNG


In [6]:
len(cohort_df.CANCER_TYPE.unique())
cohort_df.CANCER_TYPE.unique()

array(['BLCA', 'EGC', 'LMS', 'STAD', 'MEL', 'AML', 'HCC', 'UCEC', 'EPM',
       'COADREAD', 'GB', 'ANSC', 'RCC', 'OS', 'BCC', 'RMS', 'PRAD', 'ACC',
       'WDTC', 'ES', 'UTUC', 'GIST', 'LIPO', 'MBL', 'ANGS', 'GBC', 'ALL',
       'LUNG', 'BRCA', 'RBL', 'CLLSLL', 'BLADDER', 'SCLC', 'NSCLC',
       'STOMACH', 'SKCM', 'NETNOS', 'SOFT_TISSUE', 'SARCNOS', 'COAD',
       'HNSC', 'SIC', 'CHOL', 'ESCC', 'DLBCLNOS', 'GBM', 'HGGNOS', 'NBL',
       'ESCA', 'CSCC', 'SKIN', 'LUAD', 'CEAD', 'PLMESO', 'LGGNOS', 'PAAD',
       'OVT', 'LUSC', 'SACA', 'CESC', 'CCRCC', 'PANCREAS', 'ACYC', 'MT',
       'PROSTATE', 'READ', 'PCM', 'BL', 'VULVA', 'PAST', 'ATRT', 'PANET',
       'NHL', 'CHRCC', 'MDS', 'NPC', 'LIHB', 'WT', 'MGCT', 'UCS', 'UM',
       'THYM', 'PRCC', 'PGNG'], dtype=object)

# Get data
__I need:__
* Clustering scores of a given genes across all cohorts
* Cluster of a given gene across all cohorts
* Missense mutations probability of a given genes across all cohorts

In [7]:
o3d_output_dir

'/workspace/projects/clustering_3d/o3d_analysys/datasets//output/cancer_202404/o3d_output/human_mane_raw/run_2024-07-01_16-04-14'

In [48]:
def get_o3d_result(cohorts_df, o3d_output_dir):

    lst_dfs = []

    for cohort in cohorts_df["COHORT"].values:
        cohort = "TCGA_WXS_SOFT_TISSUE" if cohort == "TCGA_WXS_SARC" else cohort
        
        path_csv = f"{o3d_output_dir}/{cohort}/{cohort}.3d_clustering_pos.csv"
        path_genes_csv = f"{o3d_output_dir}/{cohort}/{cohort}.3d_clustering_genes.csv"
        if os.path.exists(path_csv) and os.path.exists(path_genes_csv):

            # Get detected genes
            df_genes_cohort = pd.read_csv(path_genes_csv)
            df_genes_cohort = df_genes_cohort[df_genes_cohort["C_gene"] == 1] 
            genes_detected = df_genes_cohort.apply(lambda x: x.Gene, axis=1).values

            if len(genes_detected) > 0:
                # Get per-residue result
                df_cohort = pd.read_csv(path_csv)
                cols = ["Gene", "Uniprot_ID", "F", "Pos", "Mut_in_gene", "Score_obs_sim", "C", "C_ext", 'Cancer', 'Cohort'] #"Mut_in_res", "Mut_in_vol", "pval"
                df_cohort = df_cohort.loc[df_cohort.Gene.isin(genes_detected), cols]
                lst_dfs.append(df_cohort)
                
        else:
            print(f"Path doesn't exist: {path_csv}")

    return pd.concat(lst_dfs)

o3d_pos_result = get_o3d_result(cohort_df, o3d_output_dir)
o3d_pos_result

/tmp/ipykernel_55141/1608040957.py:19: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df_cohort = pd.read_csv(path_csv)


Path doesn't exist: /workspace/projects/clustering_3d/o3d_analysys/datasets//output/cancer_202404/o3d_output/human_mane_raw/run_2024-07-01_16-04-14/PEDCBIOP_WXS_TALL_REL/PEDCBIOP_WXS_TALL_REL.3d_clustering_pos.csv


,Gene,Uniprot_ID,F,Pos,Mut_in_gene,Score_obs_sim,C,C_ext,Cancer,Cohort
1047,FBXW7,Q969H0,1,505,8,3.204794,1,0.0,NaN,HARTWIG_WGS_BLCA_2023
1048,FBXW7,Q969H0,1,465,8,3.409612,1,0.0,NaN,HARTWIG_WGS_BLCA_2023
1049,FBXW7,Q969H0,1,176,8,0.410806,0,NaN,NaN,HARTWIG_WGS_BLCA_2023
1050,FBXW7,Q969H0,1,280,8,0.390281,0,NaN,NaN,HARTWIG_WGS_BLCA_2023
1057,FGFR3,P22607,1,248,17,4.734346,1,0.0,NaN,HARTWIG_WGS_BLCA_2023
...,...,...,...,...,...,...,...,...,...,...
1,EPAS1,Q99814,1,531,8,5.687067,1,0.0,NaN,TCGA_WXS_PGNG
2,EPAS1,Q99814,1,530,8,6.057751,1,0.0,NaN,TCGA_WXS_PGNG
3,EPAS1,Q99814,1,532,8,6.386988,1,0.0,NaN,TCGA_WXS_PGNG
5,HRAS,P01112,1,13,18,7.463746,1,0.0,NaN,TCGA_WXS_PGNG


In [50]:
print(len(o3d_pos_result.Gene.unique()))
print(len(o3d_pos_result.Cohort.unique()))

273
135


## To do:
* I got a panda df including the per-residue result of each gene-cohort detected as significant
* Now I should get something like a dictionary gene_dict[gene] : gene[cohort] : gene_cohort[score, miss_prob, cluster, mut_in_gene]

In [ ]:
# From BLCA_NORMAL (to del)

o3d_seq_df = pd.read_table(f"{o3d_datasets}/seq_for_mut_prob.tsv")
o3d_annot_df = pd.read_csv(f"{o3d_annotations}/uniprot_feat.tsv", sep="\t")

o3d_prob = f"{deepcsa_run_dir}/oncodrive3d/run/{run_name}/{run_name}.miss_prob.processed.json"
o3d_prob = json.load(open(o3d_prob, encoding="utf-8"))

o3d_score = f"{deepcsa_run_dir}/oncodrive3d/run/{run_name}/{run_name}.3d_clustering_pos.csv"
o3d_score = pd.read_csv(o3d_score)[["Gene", "Pos", "Score", "Score_obs_sim", "pval", "C", "C_ext"]]

# Plot

In [ ]:
# Plot
# ====

def init_plot_pars():
    
    plot_pars = {"fsize"                : (20,10),
                 "hspace"               : 0.1,                    # General space between all tracks
                 "ofml_cbar_coord"      : (0.01, 0.35, 0.9, 1),   # Box to anchor coordinate for OncodriveFML color bar
                 "track_title_x_coord"  : 0.83,                   # x-coordinate (respect to protein len) for track txt title
                 "score_txt_x_coord"    : 1.13,                   # as track title but for track score txt
                 "track_title_fontsize" : 14,
                 "ylabel_fontsize"      : 13.5,
                 "xlabel_fontsize"      : 13.5,
                 "ylabel_pad"           : 38,
                 "ticksize"             : 10.5,
                 "legend_fontsize"      : 12,
                 "legend_frameon"       : True,
                 "dpi"                  : 300
                }
    plot_pars["colors"] = {"ofml"        : "viridis_r",
                           "omega_trunc" : "#FA5E32",
                           "omega_synon" : "#89E4A2",
                           "omega_miss"  : "#FABE4A",
                           "o3d_score"   : "#6DBDCC",
                           "o3d_cluster" : "skyblue",
                           "o3d_prob"    : "darkgray",
                           "frameshift"  : "#E4ACF4",
                           "inframe"     : "C5",
                           "hv_lines"    : "lightgray"      # General horizontal and vertical lines (e.g., needle plot vline)
                           }
    plot_pars["h_ratios"] = {"omega_trunc" : 0.3,
                             "omega_synon" : 0.3,
                             "omega_miss"  : 0.3,
                             "space1"      : 0.015,
                             "o3d"         : 0.5,
                             "space2"      : 0.015,
                             "ofml"        : 0.5,
                             "space3"      : 0.015,
                             "indels"      : 0.5,
                             "space4"      : 0.015,
                             "domain"      : 0.07
                            }
    
    return plot_pars


def plot_count_track(pos_track, 
                     axes, 
                     ax=0, 
                     neg_track=None, 
                     gene_len=None,
                     col_pos_track="gray",
                     col_neg_track="gray",
                     label_pos_track=None,
                     label_neg_track=None,
                     ymargin=None,
                     hv_lines="lightgray"):
    
    axes[ax].vlines(pos_track["Pos"], ymin=0, ymax=pos_track["Count"], color=hv_lines, lw=1, zorder=1, alpha=0.5)
    axes[ax].scatter(pos_track["Pos"], pos_track["Count"], color='white', zorder=3, lw=1, ec="white") # To cover the overlapping needle top part
    axes[ax].scatter(pos_track["Pos"].values, pos_track["Count"].values, color=col_pos_track, zorder=4, 
                     alpha=0.7, lw=0.1, ec="black", s=60, label=label_pos_track)
    
    if isinstance(neg_track, pd.DataFrame):
        if isinstance(gene_len, int):
            axes[ax].hlines(0, xmin=0, xmax=gene_len, color="gray", lw=0.6, zorder=1)
        axes[ax].vlines(neg_track["Pos"], ymin=-neg_track["Count"], ymax=0, color=hv_lines, lw=1, zorder=1, alpha=0.5)
        axes[ax].scatter(neg_track["Pos"], -neg_track["Count"], color='white', zorder=3, lw=1, ec="white") # To cover the overlapping needle top part
        axes[ax].scatter(pos_track["Pos"].values, -neg_track["Count"].values, color=col_neg_track, zorder=4, 
                         alpha=0.7, lw=0.1, ec="black", s=60, label=label_neg_track)
        axes[ax].set_yticklabels(abs(axes[ax].get_yticks()))
        
    if ymargin is not None:
        axes[ax].set_ylim(-np.max(neg_track["Count"])-ymargin, np.max(pos_track["Count"])+ymargin)
        
        
def add_ax_text(score, pvalue, y_text, x_text, ax, y_shift=0.2, y_adjust=0, equal_less=True):

    ax.text(x_text, y_text+(y_text*y_shift)+y_adjust, 
            fr'$\mathit{{Score}}$ = {np.round(score, 2)}', ha='center', va='center', fontsize=13.5, color="black")
    
    equal = "≤" if equal_less else "="
    ax.text(x_text, y_text-(y_text*y_shift)+y_adjust, 
            fr'$\mathit{{p}}$-value {equal} {pvalue}', ha='center', va='center', fontsize=13.5, color="black")

    
def get_transcript_ids(gene, maf_df_f, o3d_seq_df):
    
    canonical_tr = maf_df_f[maf_df_f["SYMBOL"] == gene].canonical_Feature.unique()[0]
    o3d_tr = o3d_seq_df[o3d_seq_df["Gene"] == gene].Ens_Transcr_ID.values[0]
    
    return canonical_tr, o3d_tr


def plot_gene_selection_signals(maf_trunc_count_gene,
                                maf_miss_count_gene,
                                maf_synon_count_gene,
                                frameshift_indels_count_gene,
                                inframe_indels_count_gene,
                                ofml_muts_score_gene,
                                o3d_score_gene,
                                o3d_prob_gene,
                                plot_pars,
                                domain_df=None,
                                add_track_title_text=True,
                                add_score_text=False,
                                rm_spines=False,
                                light_spines=False,
                                output_path=None):
    
    
    gene_len = len(o3d_prob_gene)
    fig, axes = plt.subplots(len(plot_pars["h_ratios"]), 1, 
                             figsize=plot_pars["fsize"], 
                             sharex=True, 
                             gridspec_kw={'hspace': plot_pars["hspace"], 
                                          'height_ratios': plot_pars["h_ratios"].values()})
    colors = plot_pars["colors"]


        
    # Oncodrive3D
    # ===========

    ax=4
    axes[ax].plot(range(1, gene_len+1), o3d_score_gene["O3D_score_norm"], zorder=2, color=colors["o3d_score"], lw=1, label="Clustering score")                       
    axes[ax].plot(range(1, gene_len+1), -o3d_prob_gene, zorder=3, color=colors["o3d_prob"], lw=1, label="Missense mut probability")     
    axes[ax].fill_between(o3d_score_gene['Pos'], 0, o3d_score_gene["O3D_score_norm"], where=(o3d_score_gene['Cluster'] == 1), 
                          color=colors["o3d_cluster"], alpha=0.3, label='Cluster', zorder=0, lw=2)
    if add_score_text:
        add_ax_text(score=np.round(o3d_score, 2), 
                    pvalue=o3d_pvalue,
                    y_text=(np.max(o3d_score_gene["O3D_score_norm"]) + np.max(o3d_prob_gene))/2, 
                    x_text=gene_len*plot_pars["score_txt_x_coord"], 
                    ax=axes[ax], 
                    y_shift=0.15,
                    y_adjust=-0.0075,
                    equal_less=True)
    if add_track_title_text:
        axes[ax].text(gene_len*plot_pars["track_title_x_coord"], 0.015, 
                      fr'$\mathbf{{3D}}$-$\mathbf{{Clustering}}$', 
                      ha='center', va='center', fontsize=plot_pars["track_title_fontsize"], color="black")
        
    axes[ax].set_ylabel('Score &\nprobability', fontsize=plot_pars["ylabel_fontsize"], rotation=45, labelpad=plot_pars["ylabel_pad"], va='center')
    axes[ax].yaxis.set_major_locator(MaxNLocator(integer=False, nbins=4))
    tick_labels = [np.round(abs(tick), 3) for tick in axes[ax].get_yticks()]
    axes[ax].set_yticklabels(tick_labels)
    axes[ax].tick_params(axis='y', labelsize=plot_pars["ticksize"])
    
    axes[ax].legend(loc="upper left", fontsize=plot_pars["legend_fontsize"], frameon=plot_pars["legend_frameon"])
    #axes[ax].legend(bbox_to_anchor=[0, 0.35, 1, 0])
    if rm_spines:
        axes[ax].spines['top'].set_visible(False)
        axes[ax].spines['right'].set_visible(False)
    elif light_spines:
        axes[ax].spines['top'].set_color(colors["hv_lines"])
        axes[ax].spines['right'].set_color(colors["hv_lines"])

        
    # OncodriveFML
    # ============

    ax=6
    axes[ax].vlines(ofml_muts_score_gene["Pos"].values, ymin=0, ymax=ofml_muts_score_gene["Count"], color=colors["hv_lines"], lw=1, zorder=1, alpha=0.5)
    fml_scatter = axes[ax].scatter(ofml_muts_score_gene["Pos"].values, ofml_muts_score_gene["Count"].values, 
                                   c=ofml_muts_score_gene["CADD_score"].values, cmap=colors["ofml"], zorder=4, 
                                   alpha=0.7, lw=0.1, ec="black", s=60, label="SNV")
    
    y_text=np.round(np.max(ofml_muts_score_gene["Count"])/2)
    if add_score_text:
        add_ax_text(score=np.round(ofml_score, 2), 
                    pvalue=ofml_pvalue,
                    y_text=y_text, 
                    x_text=gene_len*plot_pars["score_txt_x_coord"], 
                    ax=axes[ax], 
                    y_adjust=3,
                    equal_less=True)

    if add_track_title_text:
        axes[ax].text(gene_len*plot_pars["track_title_x_coord"], y_text+y_text*0.8, 
                      fr'$\mathbf{{Functional}}$ $\mathbf{{Impact}}$', 
                      ha='center', va='center', fontsize=plot_pars["track_title_fontsize"], color="black")

    inset_ax = inset_axes(axes[ax], width="10%", height="10%", loc='center left', 
                          bbox_to_anchor=plot_pars["ofml_cbar_coord"], bbox_transform=axes[ax].transAxes, borderpad=0)
    cbar = plt.colorbar(fml_scatter, cax=inset_ax, orientation='horizontal')
    cbar.set_label('Impact score', fontsize=plot_pars["legend_fontsize"])
    cbar.ax.tick_params(labelsize=plot_pars["ticksize"])

    n_max = np.max(ofml_muts_score_gene["Count"])
    j = 14.5
    i = 19.33
    axes[ax].set_ylim(-n_max/i, n_max + n_max/j)    
  
    axes[ax].set_ylabel('SNV count', fontsize=plot_pars["ylabel_fontsize"], rotation=45, labelpad=plot_pars["ylabel_pad"], va='center')
    axes[ax].tick_params(axis='y', labelsize=plot_pars["ticksize"])
    
    if rm_spines:
        axes[ax].spines['top'].set_visible(False)
        axes[ax].spines['right'].set_visible(False)
    elif light_spines:
        axes[ax].spines['top'].set_color(colors["hv_lines"])
        axes[ax].spines['right'].set_color(colors["hv_lines"])        
        

    # Frameshift enrichment
    # =====================

    ax=8
    plot_count_track(frameshift_indels_count_gene, axes, ax=ax, neg_track=inframe_indels_count_gene, gene_len=gene_len,
                     col_pos_track=colors["frameshift"], col_neg_track=colors["inframe"], label_pos_track="Frameshift", label_neg_track="Inframe",
                     ymargin=1, hv_lines=colors["hv_lines"])
    if add_score_text:
        add_ax_text(score=np.round(indels_score, 2), 
                    pvalue=np.round(indels_pvalue, 2),
                    y_text=(np.max(frameshift_indels_count_gene["Count"]) + np.max(inframe_indels_count_gene["Count"]))/2, 
                    x_text=gene_len*plot_pars["score_txt_x_coord"], 
                    ax=axes[ax], 
                    y_shift=0.24,
                    y_adjust=0.4,
                    equal_less=False)
    if add_track_title_text:
        axes[ax].text(gene_len*plot_pars["track_title_x_coord"], 3.9, 
                      fr'$\mathbf{{Frameshift}}$ $\mathbf{{enrichment}}$', 
                      ha='center', va='center', fontsize=plot_pars["track_title_fontsize"], color="black")
    axes[ax].legend(loc="upper left", fontsize=plot_pars["legend_fontsize"], frameon=plot_pars["legend_frameon"])
    axes[ax].set_ylabel('Indels count', fontsize=plot_pars["ylabel_fontsize"], rotation=45, labelpad=plot_pars["ylabel_pad"], va='center')
    
    axes[ax].yaxis.set_major_locator(MaxNLocator(integer=True, nbins=5))
    tick_labels = [abs(int(tick)) for tick in axes[ax].get_yticks()]
    axes[ax].set_yticklabels(tick_labels)
    
    if rm_spines:
        axes[ax].spines['top'].set_visible(False)
        axes[ax].spines['right'].set_visible(False)
    elif light_spines:
        axes[ax].spines['top'].set_color(colors["hv_lines"])
        axes[ax].spines['right'].set_color(colors["hv_lines"])


    # Domain
    # ======
    
    if isinstance(domain_df, pd.DataFrame):
        ax=10
        domain_color_dict = {}

        for n, name in enumerate(domain_df["Description"].unique()):
            domain_color_dict[name] = f"C{n}"

        n = 0
        added_domain = []
        for i, row in domain_df.iterrows():
            if pd.Series([row["Description"], row["Begin"], row["End"]]).isnull().any():
                continue

            name = row["Description"]
            start = int(row["Begin"])
            end = int(row["End"])
            axes[ax].fill_between(range(start, end+1), -0.45, 0.45,  alpha=0.5, color=domain_color_dict[name])
            if name not in added_domain:
                y = -0.04
                axes[ax].text(((start + end) / 2)+0.5, y, name, ha='center', va='center', fontsize=10, color="black")
                added_domain.append(name)
        axes[ax].set_yticks([])  
    else:
        axes[10].remove()

        
    # Spaces
    # =====
    axes[3].remove()
    axes[5].remove()
    axes[7].remove()
    axes[9].remove()

    
    # X axes
    # ======
    axes[ax].tick_params(axis='y', labelsize=plot_pars["ticksize"])
    axes[ax].tick_params(axis='x', labelsize=plot_pars["ticksize"])
    axes[ax].set_xlabel('Protein position', fontsize=plot_pars["xlabel_fontsize"]) 

    if output_path is not None:
        print(f"Saving {output_path}")
        plt.savefig(output_path, dpi=plot_pars["dpi"], bbox_inches='tight')
    plt.show()